# LGM-VIP Data Science Internship Programme
    
 ### Advanced Level Task-2
 ### Next Word Prediction
      Using Tensorflow and Keras library train a RNN, to predict the next word. 
      Dataset Link: https://drive.google.com/file/d/1GeUzNVqiixXHnTl8oNiQ2W3CynX_lsu2/view
      

In [25]:
import tensorflow as tf
import numpy as np
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
import pickle
import heapq
import warnings as wg
wg.filterwarnings("ignore")

from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

In [4]:
# source text
file = open('1661-0.txt', encoding="utf8").read().lower()

In [5]:
print('corpus length:', len(file))

corpus length: 581888


In [7]:
file = open("1661-0.txt", "r", encoding = "utf8")
lines = []

for i in file:
    lines.append(i)
    
#print(lines)

In [8]:
data = ""

for i in lines:
    data = ' '. join(lines)
    
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
data[:360]

" Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle  This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.net   Title: The Adventures of Sherl"

In [9]:
import string

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
new_data = data.translate(translator)

new_data[:500]

' Project Gutenberg s The Adventures of Sherlock Holmes  by Arthur Conan Doyle  This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever   You may copy it  give it away or re use it under the terms of the Project Gutenberg License included with this eBook or online at www gutenberg net   Title  The Adventures of Sherlock Holmes  Author  Arthur Conan Doyle  Release Date  November 29  2002  EBook  1661  Last Updated  May 20  2019  Language  English  Charact'

In [10]:
z = []

for i in data.split():
    if i not in z:
        z.append(i)
        
data = ' '.join(z)
data[:500]

"Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle This eBook is for the use anyone anywhere at no cost and with almost restrictions whatsoever. You may copy it, give it away or re-use under terms Gutenberg License included this online www.gutenberg.net Title: Holmes Author: Release Date: November 29, 2002 [EBook #1661] Last Updated: May 20, 2019 Language: English Character set encoding: UTF-8 *** START OF THIS PROJECT GUTENBERG EBOOK THE ADVENTURES SHERLOCK HOLMES Prod"

# Word Tokenization

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function.
pickle.dump(tokenizer, open('tokenizer1.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

[838, 3083, 56, 322, 57, 1523, 15, 95, 839, 3084]

In [12]:
vocab_size = len(tokenizer.word_index) + 1

In [13]:
sequences = []

for i in range(1, len(sequence_data)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  17678


array([[ 838, 3083],
       [3083,   56],
       [  56,  322],
       [ 322,   57],
       [  57, 1523],
       [1523,   15],
       [  15,   95],
       [  95,  839],
       [ 839, 3084],
       [3084, 3085]])

In [14]:
X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)

print("The Data is: ", X[:5])
print("The responses are: ", y[:5])


The Data is:  [ 838 3083   56  322   57]
The responses are:  [3083   56  322   57 1523]


In [15]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## RNN MODEL

In [16]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 10)             89310     
                                                                 
 lstm (LSTM)                 (None, 1, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 8931)              8939931   
                                                                 
Total params: 22,078,241
Trainable params: 22,078,241
Non-trainable params: 0
_________________________________________________________________


In [19]:
## call back


checkpoint = ModelCheckpoint("nextword1.h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword1'
tensorboard_Visualization = TensorBoard(log_dir=logdir)

In [22]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001))

In [27]:
model.fit(X, y, epochs=55, batch_size=150, callbacks=[checkpoint, reduce, tensorboard_Visualization])

Epoch 1/55
118/118 [==============================] - ETA: 0s - loss: 2.4565
Epoch 00001: loss improved from 2.67389 to 2.45648, saving model to nextword1.h5
118/118 [==============================] - 99s 835ms/step - loss: 2.4565 - lr: 0.0010
Epoch 2/55
118/118 [==============================] - ETA: 0s - loss: 2.2583
Epoch 00002: loss improved from 2.45648 to 2.25833, saving model to nextword1.h5
118/118 [==============================] - 101s 857ms/step - loss: 2.2583 - lr: 0.0010
Epoch 3/55
118/118 [==============================] - ETA: 0s - loss: 2.1978
Epoch 00004: loss improved from 2.22229 to 2.19776, saving model to nextword1.h5
118/118 [==============================] - 82s 699ms/step - loss: 2.1978 - lr: 0.0010
Epoch 5/55
118/118 [==============================] - ETA: 0s - loss: 2.1789
Epoch 00005: loss improved from 2.19776 to 2.17893, saving model to nextword1.h5
118/118 [==============================] - 85s 717ms/step - loss: 2.1789 - lr: 0.0010
Epoch 6/55
118/118 [===

118/118 [==============================] - 62s 524ms/step - loss: 1.8268 - lr: 0.0010
Epoch 41/55
118/118 [==============================] - ETA: 0s - loss: 1.8200
Epoch 00041: loss improved from 1.82676 to 1.81998, saving model to nextword1.h5
118/118 [==============================] - 57s 487ms/step - loss: 1.8200 - lr: 0.0010
Epoch 42/55
118/118 [==============================] - ETA: 0s - loss: 1.8211
Epoch 00043: loss did not improve from 1.81998
118/118 [==============================] - 57s 482ms/step - loss: 1.8211 - lr: 0.0010
Epoch 44/55
118/118 [==============================] - ETA: 0s - loss: 1.8157
Epoch 00044: loss improved from 1.81998 to 1.81565, saving model to nextword1.h5
118/118 [==============================] - 53s 451ms/step - loss: 1.8157 - lr: 0.0010
Epoch 45/55
118/118 [==============================] - ETA: 0s - loss: 1.8110
Epoch 00045: loss improved from 1.81565 to 1.81100, saving model to nextword1.h5
118/118 [==============================] - 53s 453ms/s

In [30]:
model = load_model('nextword1.h5')
tokenizer = pickle.load(open('tokenizer1.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue



Enter your line: loss improved
['loss', 'improved']
insinuating
